🎯 Objectif
Réduire le bruit d’une image à l’aide :
du filtre de moyenne (mean filter)
du filtre médian (median filter)
Ces deux techniques sont des filtres de lissage utilisés en traitement d’image pour supprimer le bruit tout en préservant au mieux les détails.

Filtre de Moyenne (Mean Filter)
Principe : Remplace chaque pixel par la moyenne des pixels voisins

Effet : Réduction du bruit, mais flou des contours

Noyau typique : Matrice 3×3 avec valeurs 1/9

Filtre Médian (Median Filter)
Principe : Remplace chaque pixel par la médiane des pixels voisins

Effet : Supprime le bruit impulsionnel (sel et poivre) mieux que le filtre moyenne

Avantage : Préserve mieux les contours


Critère	Filtre Moyenne	Filtre Médian
Bruit gaussien	Bon	Moyen
Bruit sel/poivre	Moyen	Excellent
Préservation contours	Faible	Bonne
Vitesse	Rapide	Plus lent

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.util import random_noise

def add_noise_to_image(image, noise_type='gaussian'):
    """
    Ajouter du bruit à une image pour la démonstration
    """
    if noise_type == 'gaussian':
        noisy_img = random_noise(image, mode='gaussian', var=0.01)
    elif noise_type == 'salt_pepper':
        noisy_img = random_noise(image, mode='s&p', amount=0.05)
    elif noise_type == 'speckle':
        noisy_img = random_noise(image, mode='speckle', var=0.01)
    
    return (noisy_img * 255).astype(np.uint8)

def mean_filter(image, kernel_size=3):
    """
    Appliquer un filtre de moyenne
    """
    # Créer un noyau de moyenne
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size * kernel_size)
    
    # Appliquer le filtre
    filtered_image = cv2.filter2D(image, -1, kernel)
    
    return filtered_image

def median_filter(image, kernel_size=3):
    """
    Appliquer un filtre médian
    """
    filtered_image = cv2.medianBlur(image, kernel_size)
    return filtered_image

def compare_filters(original_image, noisy_image):
    """
    Comparer les résultats des différents filtres
    """
    # Appliquer les filtres
    mean_filtered_3x3 = mean_filter(noisy_image, 3)
    mean_filtered_5x5 = mean_filter(noisy_image, 5)
    
    median_filtered_3x3 = median_filter(noisy_image, 3)
    median_filtered_5x5 = median_filter(noisy_image, 5)
    
    # Calculer les métriques de qualité
    def calculate_psnr(original, filtered):
        mse = np.mean((original - filtered) ** 2)
        if mse == 0:
            return float('inf')
        return 20 * np.log10(255.0 / np.sqrt(mse))
    
    # Afficher les résultats
    plt.figure(figsize=(15, 10))
    
    images = [
        original_image, noisy_image,
        mean_filtered_3x3, mean_filtered_5x5,
        median_filtered_3x3, median_filtered_5x5
    ]
    
    titles = [
        'Image Originale', 
        f'Image Bruitée\n(PSNR: {calculate_psnr(original_image, noisy_image):.2f} dB)',
        f'Filtre Moyenne 3x3\n(PSNR: {calculate_psnr(original_image, mean_filtered_3x3):.2f} dB)',
        f'Filtre Moyenne 5x5\n(PSNR: {calculate_psnr(original_image, mean_filtered_5x5):.2f} dB)',
        f'Filtre Médian 3x3\n(PSNR: {calculate_psnr(original_image, median_filtered_3x3):.2f} dB)',
        f'Filtre Médian 5x5\n(PSNR: {calculate_psnr(original_image, median_filtered_5x5):.2f} dB)'
    ]
    
    for i in range(6):
        plt.subplot(2, 3, i+1)
        plt.imshow(images[i], cmap='gray')
        plt.title(titles[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'mean_3x3': mean_filtered_3x3,
        'mean_5x5': mean_filtered_5x5,
        'median_3x3': median_filtered_3x3,
        'median_5x5': median_filtered_5x5
    }

# Programme principal
def main():
    # Charger une image (remplacer par votre chemin)
    # image = cv2.imread('votre_image.jpg', cv2.IMREAD_GRAYSCALE)
    
    # Ou créer une image de test
    image = cv2.imread('images/cat_large.jpg', cv2.IMREAD_GRAYSCALE)
    if image is None:
        # Créer une image synthétique si l'image n'est pas trouvée
        image = np.ones((300, 300), dtype=np.uint8) * 128
        cv2.rectangle(image, (50, 50), (150, 150), 200, -1)
        cv2.circle(image, (200, 200), 40, 100, -1)
    
    # Ajouter différents types de bruit
    print("Choisissez le type de bruit:")
    print("1. Bruit gaussien")
    print("2. Bruit sel et poivre")
    print("3. Bruit speckle")
    
    choice = input("Entrez votre choix (1-3): ")
    
    noise_types = {
        '1': 'gaussian',
        '2': 'salt_pepper', 
        '3': 'speckle'
    }
    
    noise_type = noise_types.get(choice, 'gaussian')
    noisy_image = add_noise_to_image(image, noise_type)
    
    # Comparer les filtres
    filtered_images = compare_filters(image, noisy_image)
    
    # Sauvegarder les résultats
    cv2.imwrite('image_originale.png', image)
    cv2.imwrite('image_bruitee.png', noisy_image)
    cv2.imwrite('filtre_moyenne_3x3.png', filtered_images['mean_3x3'])
    cv2.imwrite('filtre_median_3x3.png', filtered_images['median_3x3'])
    
    print("Images sauvegardées:")
    print("- image_originale.png")
    print("- image_bruitee.png") 
    print("- filtre_moyenne_3x3.png")
    print("- filtre_median_3x3.png")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'skimage'

In [ ]:
import cv2
import numpy as np

# Charger l'image
image = cv2.imread('images/cat_large.jpg')

# Convertir en niveaux de gris si nécessaire
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 1. FILTRE DE MOYENNE (Mean Filter)
# Noyau 3x3
kernel_3x3 = np.ones((3, 3), np.float32) / 9.0
mean_filtered_3x3 = cv2.filter2D(gray_image, -1, kernel_3x3)

# Noyau 5x5  
kernel_5x5 = np.ones((5, 5), np.float32) / 25.0
mean_filtered_5x5 = cv2.filter2D(gray_image, -1, kernel_5x5)

# 2. FILTRE MÉDIAN (Median Filter)
median_filtered_3x3 = cv2.medianBlur(gray_image, 3)
median_filtered_5x5 = cv2.medianBlur(gray_image, 5)

# Afficher les résultats
cv2.imshow('Original', gray_image)
cv2.imshow('Moyenne 3x3', mean_filtered_3x3)
cv2.imshow('Moyenne 5x5', mean_filtered_5x5)
cv2.imshow('Median 3x3', median_filtered_3x3)
cv2.imshow('Median 5x5', median_filtered_5x5)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
